In [20]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = (8.0, 8.0)
import CameraNetwork
import CameraNetwork.global_settings as gs
import cv2
import Image
import ImageDraw
import time
from tqdm import tqdm_notebook

In [4]:
c = CameraNetwork.CLIclient()
proxy_params = CameraNetwork.retrieve_proxy_parameters()
c.start(proxy_params)
time.sleep(3)
SERVERS = c.client_instance.servers
print SERVERS

C:\Users\User\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


['108', '106', '107']


In [22]:
df = c.query('108', '02/04/2017')

In [23]:
video = cv2.VideoWriter('video_2017_02_04.avi', -1, 6, (301, 301))

for ind in tqdm_notebook(df.loc[(slice(None), '2'), :].index):
    img, _ = c.seek('108', ind[0], ind[1], True, 301)
    img = Image.fromarray(img)
    draw = ImageDraw.Draw(img)
    draw.text(
        (0, 0),
        ind[0].time().strftime("%H:%M:%S"),
        (255,255,255)
    )

    video.write(cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))
                             
video.release()